# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

###### Цель исследования:
Рассчитать, в каком из трех регионов добычи нефти развивать прибыльнее всего.

###### Ход исследования:
1. Открыть и изучить предоставленные данные и подготовить их к исследованию.
2. Обучить и проверить модель для каждого региона.
3. Рассчитать достаточный объём сырья для безубыточной разработки новой скважины. 
4. Рассчитать прибыль по выбранным скважинам и предсказаниям модели.
5. Посчитать риски и прибыль для каждого региона.
6. Сделать вывод. Предложить регион для разработки скважин и обосновать выбор.

## Загрузка и подготовка данных

Подготовим библиотеки

In [1]:
import pandas as pd
import scipy.stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState


Загрузим данные и изучим их.

1. Изучим первый датасет.

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
print(data_0.head(10))
print(data_0.info())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


На первый взгляд данные в норме, соответствуют заявленным в документации, пропусков в них нет. Проверим на наличие явных и неявных дубликатов.

In [3]:
data_0.duplicated().sum()

0

In [4]:
data_0[data_0['id'].duplicated(keep=False)]

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
16633,fiKDv,0.157341,1.028359,5.585586,95.817889
21426,Tdehs,0.829407,0.298807,-0.049563,96.035308
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205


In [5]:
data_0[data_0['id'].duplicated()].count()

id         10
f0         10
f1         10
f2         10
product    10
dtype: int64

В датасете обнаружены дубликаты - записи о некоторых скважинах встречаются в данных дважды, таких данных немного (всего 10 строк). Удалим их и оставим только уникальные строки.

In [6]:
data_0 = data_0.drop_duplicates(subset='id')
data_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99990 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99990 non-null  object 
 1   f0       99990 non-null  float64
 2   f1       99990 non-null  float64
 3   f2       99990 non-null  float64
 4   product  99990 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


Первый датасет изучен, дубликаты устранены.

2. Загрузим и подготовим второй датасет.

In [7]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
print(data_1.head(10))
print(data_1.info())

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
5  HHckp  -3.327590  -2.205276  3.003647   84.038886
6  h5Ujo -11.142655 -10.133399  4.002382  110.992147
7  muH9x   4.234715  -0.001354  2.004588   53.906522
8  YiRkx  13.355129  -0.332068  4.998647  134.766305
9  jG6Gi   1.069227 -11.025667  4.997844  137.945408
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memor

Данные совпадают с заявленными в документации. Пропусков нет. Проверим на дубликаты.

In [8]:
data_1.duplicated().sum()

0

In [9]:
data_1[data_1['id'].duplicated(keep=False)]

,id,f0,f1,f2,product
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [10]:
data_1[data_1['id'].duplicated()].count()

id         4
f0         4
f1         4
f2         4
product    4
dtype: int64

In [11]:
data_1 = data_1.drop_duplicates(subset='id')
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99996 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99996 non-null  object 
 1   f0       99996 non-null  float64
 2   f1       99996 non-null  float64
 3   f2       99996 non-null  float64
 4   product  99996 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


В ходе исследования данных были обнаружены 4 неявных дубликата - 4 записи с повторяющимся id скважин. Удалили их, как и в первом датасете, оставили только данные о скважинах с уникальным id.

3. Аналогично изучим третий датасет.

In [12]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
print(data_2.head(10))
print(data_2.info())

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746
5  LzZXx -0.758092  0.710691  2.585887   90.222465
6  WBHRv -0.574891  0.317727  1.773745   45.641478
7  XO8fn -1.906649 -2.458350 -0.177097   72.480640
8  ybmQ5  1.776292 -0.279356  3.004156  106.616832
9  OilcN -1.214452 -0.439314  5.922514   52.954532
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


Данные совпадают с заявленными, пропусков нет, проверим на дубликаты.

In [13]:
data_2.duplicated().sum()

0

In [14]:
data_2[data_2['id'].duplicated(keep=False)]

,id,f0,f1,f2,product
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


In [15]:
data_2[data_2['id'].duplicated()].count()

id         4
f0         4
f1         4
f2         4
product    4
dtype: int64

In [16]:
data_2 = data_2.drop_duplicates(subset='id')
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99996 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99996 non-null  object 
 1   f0       99996 non-null  float64
 2   f1       99996 non-null  float64
 3   f2       99996 non-null  float64
 4   product  99996 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


Были обнаружены 4 дубликата с повторяющимся индентификатором скважин. Удалили их, оставив только уникальные скважины.

Вывод: данные во всех трех датасетах были изучены, они действительно содержат информацию, заявленную в документах. Пропусков обнаружено не было. Во всех трех датасетах были обнаружены дубликаты некоторых скважин, их количество оказалось несущественно мало по отношению ко всему объему данных, поэтому было принято решение их удалить без вреда для модели и ее качества, оставив только уникальные данные.

## Обучение и проверка модели

1. Разобъем каждый датасет на обучающую и валидационную выборки в соотношении 75:25.

In [17]:
#создадим две переменные, содержащие признаки и целевой признак, из признаков также исключим столбец с id скважины, 
#для обучения модели оннам не нужен.
features_0 = data_0.drop(['id', 'product'], axis=1)
target_0 = data_0['product']

#разобъем датасет на обучающую и валидационную выборки
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

print(features_train_0.shape)
print(target_train_0.shape)

print(features_valid_0.shape)
print(target_valid_0.shape)

(74992, 3)
(74992,)
(24998, 3)
(24998,)


In [18]:
#создадим две переменные, содержащие признаки и целевой признак, из признаков также исключим столбец с id скважины, 
#для обучения модели оннам не нужен.
features_1 = data_1.drop(['id', 'product'], axis=1)
target_1 = data_1['product']

#разобъем датасет на обучающую и валидационную выборки
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

print(features_train_1.shape)
print(target_train_1.shape)

print(features_valid_1.shape)
print(target_valid_1.shape)

(74997, 3)
(74997,)
(24999, 3)
(24999,)


In [19]:
#создадим две переменные, содержащие признаки и целевой признак, из признаков также исключим столбец с id скважины, 
#для обучения модели оннам не нужен.
features_2 = data_2.drop(['id', 'product'], axis=1)
target_2 = data_2['product']

#разобъем датасет на обучающую и валидационную выборки
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

print(features_train_2.shape)
print(target_train_2.shape)

print(features_valid_2.shape)
print(target_valid_2.shape)

(74997, 3)
(74997,)
(24999, 3)
(24999,)


2. Обучим модель и сделаем предсказания на валидационной выборке. Для обучения будем использовать Линейную регрессию по условию задачи.

In [20]:
model_0 = LinearRegression()
#Обучим модель на тренировочной выборке
model_0.fit(features_train_0, target_train_0)
#получим предсказания модели на валидационной выборке и сохраним их в отдельную переменную
predicted_valid_0 = pd.Series(model_0.predict(features_valid_0), index=features_valid_0.index)
# Сделаем расчет метрик MSE  и RMSE
mse_0 = mean_squared_error(target_valid_0, predicted_valid_0)
rmse_0 = mse_0 ** 0.5
#Посчитаем средний запас предсказанного сырья
mean_predicted_valid_0 = predicted_valid_0.mean()

print("Первый регион")
print("MSE = ", mse_0)
print("RMSE = ", rmse_0)
print("Средний запас предсказанного сырья", mean_predicted_valid_0)


Первый регион
MSE =  1432.8895312377324
RMSE =  37.853527328872964
Средний запас предсказанного сырья 92.78915638280621


In [21]:
model_1 = LinearRegression()
#Обучим модель на тренировочной выборке
model_1.fit(features_train_1, target_train_1)
#получим предсказания модели на валидационной выборке и сохраним их в отдельную переменную
predicted_valid_1 = pd.Series(model_1.predict(features_valid_1), index=features_valid_1.index)
# Сделаем расчет метрик MSE  и RMSE
mse_1 = mean_squared_error(target_valid_1, predicted_valid_1)
rmse_1 = mse_1 ** 0.5
#Посчитаем средний запас предсказанного сырья
mean_predicted_valid_1 = predicted_valid_1.mean()

print("Второй регион")
print("MSE = ", mse_1)
print("RMSE = ", rmse_1)
print("Средний запас предсказанного сырья", mean_predicted_valid_1)


Второй регион
MSE =  0.7957697318650314
RMSE =  0.892059264771703
Средний запас предсказанного сырья 69.17831957030432


In [22]:
model_2 = LinearRegression()
#Обучим модель на тренировочной выборке
model_2.fit(features_train_2, target_train_2)
#получим предсказания модели на валидационной выборке и сохраним их в отдельную переменную
predicted_valid_2 = pd.Series(model_2.predict(features_valid_2), index=features_valid_2.index)
# Сделаем расчет метрик MSE  и RMSE
mse_2 = mean_squared_error(target_valid_2, predicted_valid_2)
rmse_2 = mse_2 ** 0.5
#Посчитаем средний запас предсказанного сырья
mean_predicted_valid_2 = predicted_valid_2.mean()

print("Третий регион")
print("MSE = ", mse_2)
print("RMSE = ", rmse_2)
print("Средний запас предсказанного сырья", mean_predicted_valid_2)

Третий регион
MSE =  1606.0738119304278
RMSE =  40.07585073246016
Средний запас предсказанного сырья 94.86572480562035


Вывод: Во втором регионе модель сделала самые качественные предсказания, RMSE этой модели оказалось даже меньше 1 - 0.89. В первом и третьем регионах эта метрика получилась очень высокой - 37,9 и 40 , т.е. модель в среднем ошибается на 37,9 и 40 тыс. баррелей соответственно.

Однако, средний запас предсказанного сырья во втором регионе оказался самым низким - 69.2 тыс. баррелей, в отличие от первого и третьего регионов - 92,79 и 94,87 соответственно. 

Кажется, что развивать первый и третий регионы будет прибыльнее, если смотреть только на объемы предсказанного сырья, с другой стороны, опираясь на качество модели и показатель RMSE есть большая вероятность не получить ожидаемых объемов. Тогда как во втором месторождении предсказания куда более стабильны.

## Подготовка к расчёту прибыли

1. В условии задачи нам даны некоторые ключевые значения для расчетов. Сохраним их в переменных

In [23]:
#Бюджет на разработку скважин в регионе — 10 млрд рублей
budget_rub = 10000000000
#Общее количество исследуемых точек
research_field_num = 500
#Количество лучших точек для разработки
best_field_num = 200
#Доход с каждой единицы продукта составляет 450 тыс. рублей
income_rub = 450000
#Максимальная вероятность убытков - 2.5%
max_loss = 0.025

2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе. 

In [24]:
budget_per_field = budget_rub / best_field_num #рассчитаем бюджет, выделенный на разработку 1 скважины
min_product_amount = budget_per_field / income_rub #Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины
min_product_amount

111.11111111111111

In [25]:
print("Минимальный объём сырья для безубыточной разработки новой скважины:", min_product_amount)
print("Средний запас в первом регионе:", target_valid_0.mean())
print("Средний запас во втором регионе:", target_valid_1.mean())
print("Средний запас в третьем регионе:", target_valid_2.mean())

Минимальный объём сырья для безубыточной разработки новой скважины: 111.11111111111111
Средний запас в первом регионе: 92.15820490940044
Средний запас во втором регионе: 69.18604400957675
Средний запас в третьем регионе: 94.7851093536914


Исходя из полученных данных, минимальный объем сырья для прибыльной разработки скважины - 111,11 тыс. баррелей, ни один из регионов не достиг данной цифры. На первы взгляд кажется, что все три региона будут убыточными, но мы взяли среднее по всем скважинам. Нужно выбрать 200 скважин с максимальной прибыльностью, чтобы более точно оценить месторождения.

3. Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [26]:
def find_profit(target_valid, predicted_valid):
    predicted = predicted_valid.sort_values(ascending = False).head(200) #выбираем 200 скважин с наибольшим предсказанным количеством сырья 
    real_product_amount = target_valid[predicted.index].sum() #суммируем реальный объем сырья этих предсказанных 200 скважин
    profit = (income_rub * real_product_amount - budget_rub) / 1000000000 #вычисляем прибыль с этих 200 скважин (млрд)
    return profit
    

In [27]:
find_profit(target_valid_0, predicted_valid_0)

3.3651872377002867

In [28]:
find_profit(target_valid_1, predicted_valid_1)

2.4150866966815108

In [29]:
find_profit(target_valid_2, predicted_valid_2)

2.5012838532820627

Исходя из расчетов, наиболее прибыльным оказался первый регион - его прибыль свыше 3-х млрд рублей, тогда как в двух других регионах это около 2.5 млрд.

## Расчёт прибыли и рисков 

Посчитаем риски и прибыль для каждого региона:

In [34]:
state = RandomState(12345)
def bootstrap(target, predictions):
    values = []
    lost = 0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        profit = find_profit(target_subsample, probs_subsample)
        if profit < 0:
            lost +=1
        values.append(profit)
        
    values = pd.Series(values)
    lower = values.quantile(max_loss)
    upper = values.quantile(1-max_loss)
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    mean = values.mean()
    
    print("Средняя выручка:", mean)
    print("Доверительный интервал:", confidence_interval)
    print("Вероятность убытка:", lost / 1000)

In [35]:
bootstrap(target_valid_0, predicted_valid_0)

Средняя выручка: 0.5856522724429932
Доверительный интервал: (0.5659593801225693, 0.6053451647634172)
Вероятность убытка: 0.028


In [36]:
bootstrap(target_valid_1, predicted_valid_1)

Средняя выручка: 0.6851300146691893
Доверительный интервал: (0.668434083841639, 0.7018259454967396)
Вероятность убытка: 0.001


In [37]:
bootstrap(target_valid_2, predicted_valid_2)

Средняя выручка: 0.5231644372384128
Доверительный интервал: (0.5031849692124908, 0.5431439052643348)
Вероятность убытка: 0.058


Вывод: Исходя из полученных данных, единственный регион, который проходит по оценки рисков - это второй регион, где вероятность убытка менее 2,5%.Кроме того, средняя выручка во втором регионе выше, чем в остальных. Таким образом, нужно выбрать именно этот регион для развития.

## Общий вывод

#### Этап 1. Изучение и предобработка данных.
В ходе данного этапа были открыты файли и изучены данные в них. Данные были проверены на пропуски и дубликаты. Были удалены строки с дублирующимися индентификаторами скважин.


#### Этап 2. Обучение и проверка модели для каждого региона.
Данные были разбиты на обучающую и валидационную выборку в соотношении 75:25. Модель была обучена и протестирована на обучающих данных. Лучше всего модель показала себя на данных второго региона, предскаазния были наиболее точными, а метрика RMSE наиболее низкой.

#### Этап 3. Расчет прибыли
Все ключевые значения для расчетов были сохранены в отдельные переменные. 
Был рассчитан достаточный объём сырья для безубыточной разработки новой скважины, он составил 111,11 тыс. баррелей.
Была написана функция для расчёта прибыли по выбранным скважинам и предсказаниям модели.

#### Этап 4. Посчитаны риски и прибыль для каждого региона
Исходя из расчетов для развития нужно выбрать второй регион, т.к. Вероятность убытков там значительно ниже, чем в других регионах, также у этого региона самая высокая средняя выручка, и относительно небольшой доверительный интервал.
